In [40]:
import numpy as np
import miditoolkit
import math

In [ ]:
# midi folder
# midi file -> quantize
# quantized midi file -> graph

In [17]:
midi_file = "/Users/kshitij1/Documents/GitHub/Clarinet/Data/Midi/Collection/Original Collection/001.mid"

In [25]:
mid_in=miditoolkit.midi.parser.MidiFile(midi_file)    
notes = mid_in.instruments[2].notes
notes.sort(key=lambda x: x.start)

In [26]:
# start = notes[0].start

# for note in notes:
#     note.start = note.start - start
#     note.end = note.end - start

# notes[:5]

In [27]:
def quantize(notes, planck=1):
    start = notes[0].start

    for note in notes:
        note.start = note.start - start
        note.end = note.end - start
    
    notes.sort(key=lambda x: x.end)
    end = notes[-1].end
    notes.sort(key=lambda x: x.start)

    quantized_notes = []
    for i in range(int(end/planck)):
        quantized_notes.append([])
    
    for note in notes:
        cur_start = note.start
        cur_end = note.end
        first_idx = int(cur_start/planck)
        last_idx = int(cur_end/planck)
        pitch = note.pitch
        vel = note.velocity
        for i in range(first_idx, last_idx):
            quantized_notes[i].append((pitch,vel))
    
    return quantized_notes

In [28]:
qn = quantize(notes, planck=1)

In [32]:
def weight(note1, note2):
    # have tuples in notei
    return 1

In [35]:
class Graph:
    def __init__(self, box_list):
        self.box_list = box_list
        self.layers = [[] for i in range(len(box_list))]
        # each layer will contain [ [(pitch,velocity,[list of weights corresponding next layer])], ... ]
    
    def create_graph(self):
        for i in range(len(self.box_list)-1):
            box1 = self.box_list[i]
            box2 = self.box_list[i+1]
            for note1 in box1:
                weight_list = []
                for note2 in box2:
                    weight_list.append(weight(note1,note2))
                self.layers[i].append([note1, weight_list])
            

In [36]:
G = Graph(qn)

In [37]:
G.create_graph()

In [44]:
def makeMelodicTransitionMat(midi_folder):
    mat = np.zeros((128,128))
    for f in midi_folder:
        mid_in=miditoolkit.midi.parser.MidiFile(f)    
        notes = mid_in.instruments[0].notes
        qn = quantize(notes, planck=1)
        for i in range(len(qn)-1):
            note1 = qn[i][0]
            note2 = qn[i+1][0]
            p1 = note1[0]
            p2 = note2[0]
            mat[p1][p2] += 1
    for i in range(128):
        mat[i] = mat[i]/np.sum(mat[i])
    return mat
    

In [45]:
def makeNonMelodicTransitionMat(midi_folder):
    mat = np.zeros((128,128))
    for f in midi_folder:
        mid_in=miditoolkit.midi.parser.MidiFile(f)    
        for channel in range(1,3):
            notes = mid_in.instruments[channel].notes
            qn = quantize(notes, planck=1)
            for i in range(len(qn)-1):
                box1 = qn[i]
                box2 = qn[i+1]
                for _ in range(5):
                    note1 = box1[np.random.randint(len(box1))]
                    note2 = box2[np.random.randint(len(box2))]
                    p1 = note1[0]
                    p2 = note2[0]
                    mat[p1][p2] += 1
    for i in range(128):
        mat[i] = mat[i]/np.sum(mat[i])
    return mat
    

In [ ]:
def p(pitch1, pitch2):
    return 1

def q(pitch1, pitch2):
    return 1

def weight(note1,note2,type="prob"):
    p1 = note1[0]
    p2 = note2[0]
    if type == "prob":
        return -1 * math.log(p(p1,p2)/q(p1,p2))
    return 1

In [ ]:
# ACCOUNT FOR RESTNOTE EVERYWHERE